# Linear regression: health insurance cost

## Notebook setup

Handle imports of necessary modules up-front.

In [ ]:
# Standard library imports
from pathlib import Path

# Third-party imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Custom functions for this notebook
import helper_functions as funcs

## 1. Data loading

### 1.1. Load

In [ ]:
data_url = 'https://raw.githubusercontent.com/4GeeksAcademy/linear-regression-project-tutorial/main/medical_insurance_cost.csv'
data_df = pd.read_csv(data_url, sep=',')

### 1.2. Save local copy

In [ ]:
# Make a directory for raw data
Path('../data/raw').mkdir(exist_ok=True, parents=True)

# Save a local copy of the raw data
data_df.to_parquet('../data/raw/medical-insurance-cost.parquet')

### 1.3. Inspect

In [ ]:
data_df.head()

In [ ]:
data_df.info()

## 2. EDA

### 2.1. Data composition

#### 2.1.1. Interval features

In [ ]:
interval_features=['age','bmi','children','charges']
data_df[interval_features].describe()

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(6,6))
axs = axs.flatten()

fig.suptitle('Patient feature distributions')

for i, feature in enumerate(interval_features):
    axs[i].set_title(feature)
    axs[i].hist(data_df[feature], color='black', bins=15)
    axs[i].set_ylabel('Count')

fig.tight_layout()
fig.show()

#### 2.1.2. Nominal features

In [ ]:
nominal_features=['sex','smoker','region']
level_counts = funcs.get_level_counts(data_df, nominal_features)
level_counts.head(len(level_counts))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
axs = axs.flatten()

fig.suptitle('Patient feature level counts')

for i, feature in enumerate(nominal_features):

    level_counts = data_df[feature].value_counts()

    axs[i].set_title(feature)
    axs[i].bar(list(range(len(level_counts))), level_counts, tick_label=level_counts.index, color='black')
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_ylabel('Customers')

fig.tight_layout()
fig.show()

### 2.2. Feature interactions

#### 2.2.1. Interval features vs label

In [ ]:
interval_features = ['age', 'bmi', 'children']

fig, axs = plt.subplots(1, 3, figsize=(10, 3))
axs = axs.flatten()

fig.suptitle('Interval features vs cost')

for i, feature in enumerate(interval_features):

    level_counts = data_df[feature].value_counts()

    if feature != 'children':
        axs[i].set_title(feature)
        axs[i].scatter(data_df[feature], data_df['charges'], color='black')
        axs[i].set_xlabel(feature)
        axs[i].set_ylabel('Charges')

    else:
        sns.boxplot(data=data_df, x=feature, y='charges', ax=axs[i])
        axs[i].set_title(feature)
        axs[i].set_xlabel(feature)
        axs[i].set_ylabel('Charges')

fig.tight_layout()
fig.show()

#### 2.2.2. Nominal features vs label

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
axs = axs.flatten()

fig.suptitle('Nominal features vs cost')

for i, feature in enumerate(nominal_features):

    sns.boxplot(data_df, x=feature, y='charges', ax=axs[i])
    axs[i].tick_params(axis='x', labelrotation=45)
    axs[i].set_xlabel(feature)
    axs[i].set_ylabel('Charges')

plt.tight_layout()
plt.show()

## 3. Data preparation

### 3.1. Train-test split

In [ ]:
training_df, testing_df = train_test_split(
    data_df,
    test_size=0.2
)

### 3.2. Feature encoding

In [ ]:
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(training_df[nominal_features])

training_df = funcs.encode_features(
    training_df,
    encoder,
    nominal_features
)

testing_df = funcs.encode_features(
    testing_df,
    encoder,
    nominal_features
)

training_df.head()

## 4. Model training

In [ ]:
results = {
    'RMSE': {},
    'R2': {}
}

### 4.1. Baseline

In [ ]:
mean_cost = training_df['charges'].mean()
rmse = root_mean_squared_error(testing_df['charges'],[mean_cost]*len(testing_df))
rsq = r2_score(testing_df['charges'],[mean_cost]*len(testing_df))

results['RMSE']['Mean cost'] = rmse
results['R2']['Mean cost'] = rsq
print(f'Mean cost model RMSE: ${rmse:.2f}')
print(f'Mean cost model R squared: ${rsq:.2f}')

In [ ]:
linear_model = LinearRegression()
linear_model.fit(training_df.drop(columns=['charges']), training_df['charges'])

predictions = linear_model.predict(testing_df.drop(columns=['charges']))
rmse = root_mean_squared_error(testing_df['charges'], predictions)
rsq = r2_score(testing_df['charges'], predictions)

results['RMSE']['Regression'] = rmse
results['R2']['Regression'] = rsq
print(f'Testing data predictions RMSE: ${rmse:.2f}')
print(f'Testing data predictions R squared: {rsq:.2f}')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(7, 3.5))
axs = axs.flatten()

axs[0].set_title('Predicted vs actual charges')
axs[0].scatter(testing_df['charges'], predictions, color='black')
axs[0].set_xlabel('Actual charges')
axs[0].set_ylabel('Predicted charges')

axs[1].set_title('Residuals')
axs[1].scatter(predictions, predictions - testing_df['charges'], color='black')
axs[1].set_xlabel('Predicted charges')
axs[1].set_ylabel('Residuals')

plt.tight_layout()
plt.show()

## 4. Optimization

### 4.1. Feature transformations

In [ ]:
scaled_training_df = training_df.copy()
scaled_testing_df = testing_df.copy()

feature_transformer = StandardScaler().fit(training_df.drop(columns=['charges']))
scaled_training_df[feature_transformer.feature_names_in_] = feature_transformer.transform(training_df[feature_transformer.feature_names_in_])
scaled_testing_df[feature_transformer.feature_names_in_] = feature_transformer.transform(testing_df[feature_transformer.feature_names_in_])

label_transformer = StandardScaler().fit(training_df['charges'].values.reshape(-1, 1))
scaled_training_df['charges'] = label_transformer.transform(training_df['charges'].values.reshape(-1, 1))
scaled_testing_df['charges'] = label_transformer.transform(testing_df['charges'].values.reshape(-1, 1))

training_df.describe()

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(6,6))
axs = axs.flatten()

fig.suptitle('Patient feature distributions')

for i, feature in enumerate(['age', 'bmi', 'children', 'charges']):
    axs[i].set_title(feature)
    axs[i].hist(scaled_training_df[feature], color='black', bins=15)
    axs[i].set_ylabel('Count')

fig.tight_layout()
fig.show()

In [ ]:
linear_model = LinearRegression()
linear_model.fit(scaled_training_df.drop(columns=['charges']), scaled_training_df['charges'])

predictions = linear_model.predict(scaled_testing_df.drop(columns=['charges']))
predictions = label_transformer.inverse_transform(predictions.reshape(-1, 1))
labels = label_transformer.inverse_transform(scaled_testing_df['charges'].values.reshape(-1, 1))
rmse = root_mean_squared_error(labels, predictions)
rsq = r2_score(labels, predictions)

results['RMSE']['Scaled regression'] = rmse
results['R2']['Scaled regression'] = rsq
print(f'Testing data predictions RMSE: ${rmse:.2f}')
print(f'Testing data predictions R squared: {rsq:.2f}')

### 4.2. Feature engineering

In [ ]:
engineered_training_df = training_df.copy()
engineered_testing_df = testing_df.copy()

engineered_training_df['charges'] = engineered_training_df['charges'].clip(upper=50000)
engineered_training_df['age'] = engineered_training_df['age'] ** 2
engineered_training_df['children'] = 1 / (engineered_training_df['children'] + 1)

engineered_testing_df['charges'] = engineered_testing_df['charges'].clip(upper=50000)
engineered_testing_df['age'] = engineered_testing_df['age'] ** 2
engineered_testing_df['children'] = 1 / (engineered_testing_df['children'] + 1)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(6,6))
axs = axs.flatten()

fig.suptitle('Patient feature distributions')

for i, feature in enumerate(['age', 'bmi', 'children', 'charges']):
    axs[i].set_title(feature)
    axs[i].hist(engineered_training_df[feature], color='black', bins=15)
    axs[i].set_ylabel('Count')

fig.tight_layout()
fig.show()

In [ ]:
linear_model = LinearRegression()
linear_model.fit(engineered_training_df.drop(columns=['charges']), engineered_training_df['charges'])

predictions = linear_model.predict(engineered_testing_df.drop(columns=['charges']))
labels = engineered_testing_df['charges'].values.reshape(-1, 1)
rmse = root_mean_squared_error(labels, predictions)
rsq = r2_score(labels, predictions)

results['RMSE']['Engineered regression'] = rmse
results['R2']['Engineered regression'] = rsq
print(f'Testing data predictions RMSE: ${rmse:.2f}')
print(f'Testing data predictions R squared: {rsq:.2f}')

### 4.3. Synthetic features

In [ ]:
poly_transformer = PolynomialFeatures(degree=4, include_bias=False)
poly_transformer.fit(engineered_training_df.drop(columns=['charges']))
poly_training_features = poly_transformer.transform(engineered_training_df.drop(columns=['charges']))
poly_testing_features = poly_transformer.transform(engineered_testing_df.drop(columns=['charges']))

poly_training_df = pd.DataFrame(poly_training_features, columns=poly_transformer.get_feature_names_out())
poly_testing_df = pd.DataFrame(poly_testing_features, columns=poly_transformer.get_feature_names_out())

poly_training_df['charges'] = engineered_training_df['charges'].values
poly_testing_df['charges'] = engineered_testing_df['charges'].values

poly_training_df.head().transpose()


In [ ]:
linear_model = LinearRegression()
linear_model.fit(poly_training_df.drop(columns=['charges']), poly_training_df['charges'])

predictions = linear_model.predict(poly_testing_df.drop(columns=['charges']))
labels = poly_testing_df['charges'].values.reshape(-1, 1)
rmse = root_mean_squared_error(labels, predictions)
rsq = r2_score(labels, predictions)

results['RMSE']['Synthetic features regression'] = rmse
results['R2']['Synthetic features regression'] = rsq
print(f'Testing data predictions RMSE: ${rmse:.2f}')
print(f'Testing data predictions R squared: {rsq:.2f}')

### 4.4. Results

In [ ]:
plt.figure(figsize=(10, 5))
plt.barh(list(results['R2'].keys()), list(results['R2'].values()), color='black')
plt.xlabel('R-squared')
plt.title('Model performance comparison (R-squared)')
plt.show()

## 5. BONUS: split smoker/nonsmoker model

In [ ]:
split_model = funcs.SplitModel(poly_training_df, poly_testing_df)
split_model.fit()
split_model.predict()
rmse, rsq = split_model.evaluate()

results['RMSE']['Split regression'] = rmse
results['R2']['Split regression'] = rsq
print(f'Testing data predictions RMSE: ${rmse:.2f}')
print(f'Testing data predictions R squared: {rsq:.2f}')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
axs[0].barh(list(results['R2'].keys()), list(results['R2'].values()), color='black')
axs[0].set_xlabel('R-squared')
axs[0].set_title('Model performance comparison (R-squared)')

axs[1].barh(list(results['RMSE'].keys()), list(results['RMSE'].values()), color='black')
axs[1].set_xlabel('RMSE')
axs[1].set_title('Model performance comparison (RMSE)')

plt.show()